<a href="https://colab.research.google.com/github/ThuanPhong0126/PaternRecognize-project-cs338/blob/main/xlmr_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Tue Jul 13 14:33:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/drive/MyDrive/DoAnNhanDang

/content/drive/MyDrive/DoAnNhanDang


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 15.2MB/s 
     |████████████████████████████████| 3.3MB 26.3MB/s 
     |████████████████████████████████| 901kB 53.6MB/s 


In [ ]:
import torch 
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
!pip install transformers

In [ ]:
class BuildDataset(torch.utils.data.Dataset): 
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
import pandas as pd
train= pd.read_csv("/content/drive/MyDrive/DoAnNhanDang/Quora dataset/train.csv")
dev= pd.read_csv("/content/drive/MyDrive/DoAnNhanDang/Quora dataset/dev.csv")
test= pd.read_csv("/content/drive/MyDrive/DoAnNhanDang/Quora dataset/test.csv")

In [ ]:
train.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
1,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
2,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
3,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
4,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0


In [ ]:
dev.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,340617,468377,468378,How do you know if a lightning strikes your ho...,How often does lightning strike the Eiffel Tower?,0
1,89072,149711,149712,Are facials good or bad for your skin?,Are back facials good for your skin?,0
2,270535,388449,140381,What are Bust Rhymes fastest raps?,What does RAP mean?,0
3,66193,114771,114772,What's the funniest conversation you've overhe...,What is the funniest conversation you overheard?,1
4,197370,257850,108173,What is the best book to learn c#?,What are the best books to learn c programing ...,1


In [ ]:
test.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,96431,160631,17921,What's the best time to visit Bermuda?,What are some of the most amazing places to vi...,0
1,112142,183533,183534,Can the value of oil ETF USO can go to zero?,I started programming 4 years ago. But recentl...,0
2,264650,336711,381462,Can I charge my phone with a charger that outp...,Can I charge my phone with a charger that outp...,1
3,209773,314137,314138,What if Gohan entered the Hyperbolic Time Cham...,What if Gohan had went in with Goten in the hy...,1
4,65399,113510,113511,What's the difference between Super AMOLED and...,Where can I get a custom AMOLED screen made?,0


In [ ]:
train=train.fillna("No question")
dev=dev.fillna("No question")
test=test.fillna("No question")
X_train = train['question1']+" [SEP] "+train['question2']
y_train = train['is_duplicate'].values

X_dev = dev['question1']+" [SEP] "+dev['question2']
y_dev = dev['is_duplicate'].values

X_test = test['question1']+" [SEP] "+test['question2']
y_test = test['is_duplicate'].values

In [ ]:
print(len(X_train),len(y_train))
print(len(X_dev),len(y_dev))
print(len(X_test),len(y_test))

244290 244290
80000 80000
80000 80000


In [ ]:
le = LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels = 2)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [ ]:
train_encodings = tokenizer(X_train.to_list(), truncation=True, padding=True, max_length=70)
dev_encodings = tokenizer(X_dev.to_list(), truncation=True, padding=True, max_length=70)
test_encodings = tokenizer(X_test.to_list(), truncation=True, padding=True, max_length=70)

y_train_encoding = le.transform(y_train)
y_dev_encoding = le.transform(y_dev)
y_test_encoding = le.transform(y_test)

In [ ]:
train_dataset = BuildDataset(train_encodings, y_train_encoding)
dev_dataset = BuildDataset(dev_encodings, y_dev_encoding)
test_dataset = BuildDataset(test_encodings, y_test_encoding)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results_xlmr_base_org',          
    num_train_epochs=3,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=32,   
    warmup_steps=1000,                
    weight_decay=0.01,
    #no_cuda=False
)

In [ ]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=dev_dataset             
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 244290
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 22905


Step,Training Loss
500,0.545700
1000,0.447600
1500,0.420900
2000,0.390800
2500,0.369100
3000,0.364900
3500,0.357600
4000,0.353700
4500,0.340200
5000,0.333800


Saving model checkpoint to ./results_xlmr_base_org/checkpoint-500
Configuration saved in ./results_xlmr_base_org/checkpoint-500/config.json
Model weights saved in ./results_xlmr_base_org/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results_xlmr_base_org/checkpoint-1000
Configuration saved in ./results_xlmr_base_org/checkpoint-1000/config.json
Model weights saved in ./results_xlmr_base_org/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results_xlmr_base_org/checkpoint-1500
Configuration saved in ./results_xlmr_base_org/checkpoint-1500/config.json
Model weights saved in ./results_xlmr_base_org/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results_xlmr_base_org/checkpoint-2000
Configuration saved in ./results_xlmr_base_org/checkpoint-2000/config.json
Model weights saved in ./results_xlmr_base_org/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results_xlmr_base_org/checkpoint-2500
Configuration saved in ./results_xlmr_base_o

TrainOutput(global_step=22905, training_loss=0.2726305751971245, metrics={'train_runtime': 7144.3714, 'train_samples_per_second': 102.58, 'train_steps_per_second': 3.206, 'total_flos': 8.55838096948044e+16, 'train_loss': 0.2726305751971245, 'epoch': 3.0})

In [ ]:
y_pred_classify = trainer.predict(dev_dataset)
y_pred = np.argmax(y_pred_classify.predictions, axis=-1)

y_true = y_dev_encoding

#Confusion matrix
cf = confusion_matrix(y_true, y_pred)
print(cf)
print("___________________________________________________________________")
#F1 - micro
evaluation = f1_score(y_true, y_pred, average='micro')
print("F1 - micro: " + str(evaluation))
print("___________________________________________________________________")
#F1 - macro
evaluation = f1_score(y_true, y_pred, average='macro')
print("F1 - macro: " + str(evaluation))
print("___________________________________________________________________")
#F1 - weighted
evaluation = f1_score(y_true, y_pred, average='weighted')
print("F1 - weighted: " + str(evaluation))
print("___________________________________________________________________")
#F1-binary
evaluation = f1_score(y_true, y_pred, average='binary')
print("F1 - binary: " + str(evaluation))
print("___________________________________________________________________")
# accuracy
acc=accuracy_score(y_true, y_pred)
print("Accuracy:" +str(acc))

***** Running Prediction *****
  Num examples = 80000
  Batch size = 32


[[46061  4569]
 [ 3408 25962]]
___________________________________________________________________
F1 - micro: 0.9002875
___________________________________________________________________
F1 - macro: 0.8935695820700785
___________________________________________________________________
F1 - weighted: 0.9006755542672092
___________________________________________________________________
F1 - binary: 0.8668302699454099
___________________________________________________________________
Accuracy:0.9002875


In [ ]:
y_pred_classify = trainer.predict(test_dataset)
y_pred = np.argmax(y_pred_classify.predictions, axis=-1)

y_true = y_test_encoding

#Confusion matrix
cf = confusion_matrix(y_true, y_pred)
print(cf)
print("___________________________________________________________________")
#F1 - micro
evaluation = f1_score(y_true, y_pred, average='micro')
print("F1 - micro: " + str(evaluation))
print("___________________________________________________________________")
#F1 - macro
evaluation = f1_score(y_true, y_pred, average='macro')
print("F1 - macro: " + str(evaluation))
print("___________________________________________________________________")
#F1 - weighted
evaluation = f1_score(y_true, y_pred, average='weighted')
print("F1 - weighted: " + str(evaluation))
print("___________________________________________________________________")
#F1-binary
evaluation = f1_score(y_true, y_pred, average='binary')
print("F1 - binary: " + str(evaluation))
print("___________________________________________________________________")
# accuracy
acc=accuracy_score(y_true, y_pred)
print("Accuracy:" +str(acc))

***** Running Prediction *****
  Num examples = 80000
  Batch size = 32


[[45903  4555]
 [ 3313 26229]]
___________________________________________________________________
F1 - micro: 0.9016500000000001
___________________________________________________________________
F1 - macro: 0.8953189858907549
___________________________________________________________________
F1 - weighted: 0.9020496706070795
___________________________________________________________________
F1 - binary: 0.8695753074959387
___________________________________________________________________
Accuracy:0.90165


In [ ]:
trainer.save_model("/content/drive/MyDrive/DoAnNhanDang/results_xlmr_base_org/model_demo")

Saving model checkpoint to /content/drive/MyDrive/DoAnNhanDang/results_xlmr_base_org/model_demo
Configuration saved in /content/drive/MyDrive/DoAnNhanDang/results_xlmr_base_org/model_demo/config.json
Model weights saved in /content/drive/MyDrive/DoAnNhanDang/results_xlmr_base_org/model_demo/pytorch_model.bin


In [ ]:
from transformers import pipeline
model=AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/DoAnNhanDang/results_xlmr_base_org/model_demo")
tokenizer=AutoTokenizer.from_pretrained("xlm-roberta-base")
model_demo=pipeline("text-classification",model=model,tokenizer=tokenizer)

In [ ]:
model_demo("Which music do you like best? [SEP] What type of music do you like?")

[{'label': 'LABEL_1', 'score': 0.9802453517913818}]

In [ ]:
test.head(50)

,id,qid1,qid2,question1,question2,is_duplicate
0,96431,160631,17921,What's the best time to visit Bermuda?,What are some of the most amazing places to vi...,0
1,112142,183533,183534,Can the value of oil ETF USO can go to zero?,I started programming 4 years ago. But recentl...,0
2,264650,336711,381462,Can I charge my phone with a charger that outp...,Can I charge my phone with a charger that outp...,1
3,209773,314137,314138,What if Gohan entered the Hyperbolic Time Cham...,What if Gohan had went in with Goten in the hy...,1
4,65399,113510,113511,What's the difference between Super AMOLED and...,Where can I get a custom AMOLED screen made?,0
5,400738,28118,3667,How I cab use Jio sim in 3G device?,How can Jio work in a 3G phone?,1
6,251120,365157,365158,What is the purpose of buying YouTube view and...,Is it safe to buy Youtube Views?,0
7,87545,147425,12850,How did you learn English well?,What is the best way to learn to write well in...,0
8,380504,152143,512112,How do I spell etc.?,How do you spell 'kakamami'?,0
9,213800,319232,319233,How do you deal with a disease when you know t...,By what year will all diseases have a cure?,0


In [ ]:
test["question1"][38]

"Help me understand? My husband and I have a two-month long fight about a missing book. He’s sentimental value to it, but wouldn't acept another copy."

In [ ]:
test["question2"][38]

"My friend and his girlfriend had a massive fight. He considers me his only understanding friend (which I am) and he told me about it. Can someone please help (I'm confused)?"